In [12]:
import numpy as np
import pandas as pd

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CDSView, CustomJS, IndexFilter, Slider
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [6]:
n_frames = 150
data = pd.DataFrame({
    "frame": np.arange(1, n_frames + 1), "x": np.arange(1, n_frames + 1), "y": np.arange(1, n_frames + 1)[::-1]
})
data = pd.read_csv("../test.csv")
catch_frame = data[data["event"] == "pass_outcome_caught"]["frame"].unique()[0]
print(catch_frame)
data = data.loc[data.frame >= 95, ["x", "y", "frame"]]
source = ColumnDataSource(data)
view = CDSView(source=source, filters=[IndexFilter(np.arange(len(data)))])

66


In [7]:
callback_js = """
var data = source.data;
var filter = view.filters[0];
var indices = [];

for (let i = 0; i < data[frame_column].length; i++) {
    if (data[frame_column][i] == cb_obj.value) {
        indices.push(i);
    }
    // Assumes data is sorted by frame_column
    if (data[frame_column][i] > cb_obj.value) {
        break;
    }
}
filter.indices = indices;
source.change.emit();
"""

In [13]:
plot = figure(height=200)
plot.circle(
    x="x",
    y="y",
    source=source,
    view=view
)
plot.x_range.start = data["x"].min() - 1
plot.y_range.start = data["y"].min() - 1
plot.x_range.end = data["x"].max() + 1
plot.y_range.end = data["y"].max() + 1
slider = Slider(start=data.frame.min(), end=data.frame.max(), value=data.frame.min(), step=1, title="Frame")

initial_indices = np.flatnonzero(source.data["frame"] == 1)
view.filters[0].indices = initial_indices
callback = CustomJS(args={"source": source, "view": view, "frame_column": "frame"}, code=callback_js)
slider.js_on_change("value", callback)

show(column(plot, slider))

In [10]:
data.head(40)

,x,y,frame
95,61.94,22.62,95
96,62.81,22.35,96
97,63.71,22.05,97
98,64.62,21.76,98
99,65.50,21.48,99
100,66.37,21.21,100
101,67.23,20.95,101
102,68.12,20.63,102
103,68.97,20.32,103
104,69.77,19.99,104
